In [16]:
from functools import reduce
import re

import pandas as pd
#import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import FreqDist, word_tokenize, stem
from IPython.display import Markdown, HTML, display

In [2]:
default_stopwords = set(stopwords.words('portuguese'))

In [3]:
data = pd.read_csv("../output/results.csv")
data.head()

,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...


In [4]:
# Aggregate all documents together
corpus = data["text"].apply(lambda x: word_tokenize(x)).sum()

# Remove single-character tokens (mostly punctuation)
words = [word for word in corpus if len(word) > 1]

# Remove words that don't have at least one alphabetical character 
words = [word for word in words if any(c.isalpha() for c in word)]

# Lowercase all words (default_stopwords are lowercase too)
words = [word.lower() for word in words]


In [5]:
#stemmer = stem.snowball.SnowballStemmer('portuguese')

#words = [stemmer.stem(word) for word in words]

In [6]:
# calculate frequency distribution of words
fdist = FreqDist(words)

# number of words with more than 1000 occurrences
nw_1000 = len(list(filter(lambda x: x > 1000,fdist.values())))

# number of words with exactly 1 occurrence
nw_1 = len(list(filter(lambda x: x== 1, fdist.values())))

In [7]:
display(Markdown("***"))
display(Markdown("### Statistics for the *el país* collection"))
print("Total documents:              {}".format(data.shape[0]))
print("Total word occurrences:       {}".format(len(words)))
print("Vocabulary size:              {}".format(len(set(words))))
print("Words occurring > 1000 times: {}".format(nw_1000))
print("Words occurring once:         {}".format(nw_1))
display(Markdown("***"))

***

### Statistics for the *el país* collection

Total documents:              249
Total word occurrences:       188211
Vocabulary size:              24685
Words occurring > 1000 times: 19
Words occurring once:         13248


***

In [19]:
df = pd.DataFrame.from_dict(fdist, orient='index', columns=["frequency"])
df = df.sort_values(by=['frequency'], ascending=False)
df = df.rename_axis('word').reset_index()
df = df.rename_axis('r').reset_index()
df["r"] = df["r"].apply(lambda x: x + 1)
df["Pr"] = df["frequency"] / df["frequency"].sum()
df["r * Pr"] = df["r"] * df["Pr"]

display(Markdown("***"))
display(Markdown("### Most frequent *50* words from *el país* collection"))
display(HTML(df.head(50).to_html(index=False)))
display(Markdown("***"))

***

### Most frequent *50* words from *el país* collection

r,word,frequency,Pr,r * Pr
1,de,10194,0.054163,0.054163
2,que,6549,0.034796,0.069592
3,do,3463,0.018400,0.055199
4,em,3312,0.017597,0.070389
5,da,3010,0.015993,0.079963
6,um,2676,0.014218,0.085309
7,para,2301,0.012226,0.085579
8,uma,2201,0.011694,0.093555
9,com,2171,0.011535,0.103814
10,não,2023,0.010749,0.107486


***